# import

In [1]:
from pycocotools.coco import COCO

ann_file = "../annotations/instances_val2017.json"
coco = COCO(ann_file) #生成coco对象

loading annotations into memory...
Done (t=1.24s)
creating index...
index created!


# coco.imgs

In [3]:
# api: coco.imgs
ids = [str(k) for k in coco.imgs]
# print(len(ids))
print(coco.imgs[int(ids[0])])

{'license': 4, 'file_name': '000000397133.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-14 17:02:52', 'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg', 'id': 397133}


# coco.cats

In [4]:
classes = {k: v["name"] for k, v in coco.cats.items()} # 类别
print(coco.cats)

{1: {'supercategory': 'person', 'id': 1, 'name': 'person'}, 2: {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}, 3: {'supercategory': 'vehicle', 'id': 3, 'name': 'car'}, 4: {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'}, 5: {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'}, 6: {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'}, 7: {'supercategory': 'vehicle', 'id': 7, 'name': 'train'}, 8: {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'}, 9: {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'}, 10: {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'}, 11: {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'}, 13: {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'}, 14: {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'}, 15: {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'}, 16: {'supercategory': 'animal', 'id': 16, 'name': 'bird'}, 17: {'supercategory': 'animal', 'id': 17, 'name': 'cat'}, 18: {'supercat

# torch.utils.data.SubSet

In [2]:
import torch
import torchvision
from torch.utils.data import Dataset

class TensorDataset(Dataset):
    """
    TensorDataset继承Dataset, 重载了__init__(), __getitem__(), __len__()
    实现将一组Tensor数据对封装成Tensor数据集
    能够通过index得到数据集的数据，能够通过len，得到数据集大小
    """
    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor

    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index]

    def __len__(self):
        return self.data_tensor.size(0)

# 生成数据
data_tensor = torch.randn(4, 3)
target_tensor = torch.rand(4)

# 将数据封装成Dataset
tensor_dataset = TensorDataset(data_tensor, target_tensor)

# 可使用索引调用数据
print(tensor_dataset[1])
# 输出：(tensor([-1.0351, -0.1004,  0.9168]), tensor(0.4977))

# 获取数据集大小
print(len(tensor_dataset))

(tensor([ 0.9065, -0.9241, -0.9772]), tensor(0.1027))
4


In [11]:
for i in tensor_dataset:
    print(i)
indices =torch.randperm(len(tensor_dataset)).tolist()
d_train = torch.utils.data.Subset(tensor_dataset, indices)
print(indices)
for i in d_train:
    print(i)


(tensor([-0.1445,  1.4546, -0.8368]), tensor(0.5118))
(tensor([ 0.0369,  0.1086, -0.0431]), tensor(0.8527))
(tensor([0.5010, 0.1304, 0.3635]), tensor(0.5879))
(tensor([-1.6325, -0.7381, -1.4347]), tensor(0.4056))
[1, 2, 3, 0]
(tensor([ 0.0369,  0.1086, -0.0431]), tensor(0.8527))
(tensor([0.5010, 0.1304, 0.3635]), tensor(0.5879))
(tensor([-1.6325, -0.7381, -1.4347]), tensor(0.4056))
(tensor([-0.1445,  1.4546, -0.8368]), tensor(0.5118))


# del_list

In [18]:
model_urls = {
            'maskrcnn_resnet50_fpn_coco':
                'https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth',
        }
model_state_dict = torch.load('pytorch_mask_rcnn\checkpoints\maskrcnn_resnet50_fpn_coco.pth')
print('=====================================model_state_dict===================================')
print(list(model_state_dict.keys())[271:273])

pretrained_msd = list(model_state_dict.values()) #msd: model state dict
print('===============================================pretrained_msd=================================')
# print(pretrained_msd[265])
del_list = [i for i in range(265, 271)] + [i for i in range(273, 279)]
for i, del_idx in enumerate(del_list):
    pretrained_msd.pop(del_idx - i)
print('===============================================pretrained_msd after del=================================')
# print(len(pretrained_msd))

=====================================model_state_dict===================================
['backbone.fpn.inner_blocks.3.weight', 'backbone.fpn.inner_blocks.3.bias']
===============================================pretrained_msd=================================
===============================================pretrained_msd after del=================================


# resnet50 layers

In [40]:
body = torchvision.models.resnet.__dict__['resnet50'](
            pretrained=True, norm_layer=torchvision.ops.misc.FrozenBatchNorm2d)
# for name, parameter in body.named_children():
#     print(name)
# body = torch.nn.ModuleDict(d for i, d in enumerate(body.named_children()) if i < 8)
# for model in body.values():
#     print(model)
    

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
FrozenBatchNorm2d(64, eps=1e-05)
ReLU(inplace=True)
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): FrozenBatchNorm2d(64, eps=1e-05)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): FrozenBatchNorm2d(64, eps=1e-05)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): FrozenBatchNorm2d(256, eps=1e-05)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): FrozenBatchNorm2d(256, eps=1e-05)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): FrozenBatchNorm2d(64, eps=1e-05)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

In [3]:
torch.randn((2,3)).view(-1)

tensor([-0.3121,  0.7346,  0.8934,  0.4210,  0.9530, -0.6478])

In [ ]:
size = torch.tensor([1,2,3])
ratios = torch.tensor([1,2,0.5])